### Using the Greedy Tokenizer to encode text

In the first example, we use the token sets we obtained.
In the second example, we use existing token sets from BPE.

In [1]:
# some helper functions
import os
from pcatt import greedy_builder
from tqdm import tqdm
import regex
import re
import time

def process_wiki_xml(f):
    containers = []
    container = []
    for line in f:
        if line.startswith("<"):
            container = " ".join(container[1:])
            if len(container.split(" ")) >= 25:
                containers.append(container)
            container = []
            token_count = 0
            continue
        line = line.strip()
        if len(line) > 0:
            container.append(line)
    return containers

def read_cpp_res(domain):
    tokens = [bytes.fromhex(t.strip()) for t in open(f'cpp_outputs/{domain}/tokens.txt','r').read().strip().split('\n')]
    merges_per_turn = [int(x) for x in open(f'cpp_outputs/{domain}/merges.txt','r').read().strip().split('\n')]
    total = 0
    totals = []
    for m in merges_per_turn:
        total += m
        totals.append(total)
    return tokens, merges_per_turn, totals

### Example 1
We do some regex pre-processing before passing the parts to greedtok.

We can also use std::regex in cpp via:
```
gb.set_regex_pattern(pat_str)
tokenized = gb.batch_split_and_tokenize(original_texts)
```
However, note that using std::regex is slow

In [2]:
tokens, _, __ = read_cpp_res('wiki')
tokens = [bytes([i]) for i in range(256)] + tokens
gb = greedy_builder.build_greedy_tokenizer(tokens)

orig = [ x for B in 'ABCDE' for i in range(10) for x in process_wiki_xml(open(f"/data/jiapeng/wiki/cleaned/A{B}/wiki_0{i}"))]
pat_str=r"""'s|'t|'re|'ve|'m|'ll|'d| ?[\p{L}]+| ?[\p{N}]+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
processed = [[re.sub(' ','Ġ',x) for x in regex.findall(pat_str, doc)] for doc in orig]
print("Number of texts:", len(orig))

start = time.process_time()
tokenized = gb.batch_tokenize_in_parts(processed)
end = time.process_time() - start
print(sum([len(x) for x in processed])/end, "words/second")

Trie constructed
Number of texts: 73720
1192241.8657683365 words/second


In [3]:
print(orig[0][:500]) #original text
print(orig[0][-500:])

Anarchism is a political philosophy and movement that is skeptical of all justifications for authority and seeks to abolish the institutions they claim maintain unnecessary coercion and hierarchy, typically including, though not necessarily limited to, the state and capitalism. Anarchism advocates for the replacement of the state with stateless societies or other forms of free associations. As a historically left-wing movement, usually placed on the farthest left of the political spectrum, it is
ideas. The Marxist criticism of anarchism is that it has a utopian character because all individuals should have anarchist views and values. According to the Marxist view, that a social idea would follow directly from this human ideal and out of the free will of every individual formed its essence. Marxists state that this contradiction was responsible for their inability to act. In the anarchist vision, the conflict between liberty and equality was resolved through coexistence and intertwining

In [4]:
print(tokenized[0][0:100]) #encoding original text up to 100 tokens

[3138, 2127, 983, 335, 258, 1138, 6680, 263, 2943, 384, 335, 1785, 2878, 484, 261, 629, 1561, 6514, 324, 5266, 263, 2614, 716, 290, 597, 308, 427, 257, 5538, 689, 1140, 4241, 9659, 774, 3879, 289, 263, 286, 880, 8813, 44, 4452, 934, 44, 2286, 514, 6639, 3441, 290, 44, 257, 1176, 263, 2787, 983, 46, 633, 2127, 983, 6367, 628, 324, 257, 7340, 261, 257, 1176, 301, 1697, 6450, 7422, 423, 682, 3656, 261, 2382, 1956, 598, 46, 663, 258, 10230, 1411, 45, 6741, 2943, 44, 2753, 1679, 326, 257, 279, 2351, 371, 1411, 261, 257, 1138, 5516, 6637]


In [5]:
print([tokens[t].strip(b'\xc4\xa0') for t in tokenized[0][0:20]]) # token word form \xc4\xa0 is Ġ special char
print([tokens[t].strip(b'\xc4\xa0') for t in tokenized[0][-20:]])

[b'An', b'arch', b'ism', b'is', b'a', b'political', b'philosophy', b'and', b'movement', b'that', b'is', b'sk', b'ept', b'ical', b'of', b'all', b'just', b'ifications', b'for', b'authority']
[b'between', b'liber', b'ty', b'and', b'equ', b'ality', b'was', b'res', b'olved', b'through', b'co', b'ex', b'ist', b'ence', b'and', b'inter', b't', b'win', b'ing', b'.']


In [6]:
len(tokenized[0])/len(processed[0]) #token per word

1.3494058188772027

In [7]:
decoded_example = re.sub('Ġ', ' ', b''.join([tokens[c] for c in tokenized[0]]).decode('utf-8'))
print(decoded_example[:500]) #decoded tokens
print(decoded_example[-500:])

Anarchism is a political philosophy and movement that is skeptical of all justifications for authority and seeks to abolish the institutions they claim maintain unnecessary coercion and hierarchy, typically including, though not necessarily limited to, the state and capitalism. Anarchism advocates for the replacement of the state with stateless societies or other forms of free associations. As a historically left-wing movement, usually placed on the farthest left of the political spectrum, it is
ideas. The Marxist criticism of anarchism is that it has a utopian character because all individuals should have anarchist views and values. According to the Marxist view, that a social idea would follow directly from this human ideal and out of the free will of every individual formed its essence. Marxists state that this contradiction was responsible for their inability to act. In the anarchist vision, the conflict between liberty and equality was resolved through coexistence and intertwining

### Example 2

We can also use any existing BPE merge rules.

To bypass std::regex, we iterate through the text as-is without splitting.

**Time-complexity is O(W x (max token length) x lgW)**, or simply O(W^2lgW) for convenience.

We can choose to ignore regex because the regex pattern should have been imprinted onto the cover rules.

Intuitively, this algorithm detects overlapping potential covers and resolves the assignment based on the order of covering priority.

For this example, we use the merging keys/rules from cl100k

In [8]:
import tiktoken
cl100k_base = tiktoken.get_encoding("cl100k_base")
rules = list(cl100k_base._mergeable_ranks.keys())

gb = greedy_builder.build_greedy_tokenizer(rules)
astart, tstart = time.time(), time.process_time()
tokenized = gb.batch_tokenize_whole(orig)
aend, tend = time.time()-astart, time.process_time() - tstart
print(f"Absolute time: {aend:.2f} second")
print(f"Absolute time: {sum([len(x) for x in processed])/aend:.2f} words/second")
print(f"Relative time: {tend:.2f} second")
print(f"Relative time: {sum([len(x) for x in processed])/tend:.2f} words/second")
print()

Trie constructed
Absolute time: 5.29 second
Absolute time: 18425783.90 words/second
Relative time: 124.40 second
Relative time: 783086.95 words/second



On a 2.4Ghz CPU, the algorithm is processing around 750K words/s/thread.

Compared to Tiktoken, the above algorithm is _3 times slower_. Nothwithstanding Tiktoken's O(W^2) complexity, they also benefit from Rust's regex crate.

Nevertheless, the throughput is reasonable for practical applications, and may improve with future optimizations.

In [10]:
item = 50030
print(orig[item][:500]) #original text
print(tokenized[item][0:100])
print(b''.join([rules[i] for i in tokenized[item][0:100]]))
print(len(tokenized[item]) / len(processed[item]))

Kildonan—Transcona is an historical electoral division in the Canadian province of Manitoba. It was created for the 1949 provincial election, and eliminated with the 1958 provincial election. Kildonan—Transcona was located to the immediate northwest of Winnipeg, covering such suburban communities as Transcona and East Kildonan. It was won by the socialist Cooperative Commonwealth Federation (CCF) in both 1949 and 1953, but it was not considered a safe seat for the party. Russell Paulley, who ser
[42, 699, 263, 276, 2345, 3246, 91524, 374, 459, 13970, 34941, 13096, 304, 279, 12152, 17271, 315, 64340, 13, 1102, 574, 3549, 369, 279, 220, 777, 2491, 36031, 6355, 11, 323, 34373, 449, 279, 220, 777, 2970, 36031, 6355, 13, 735, 699, 263, 276, 2345, 3246, 91524, 574, 7559, 311, 279, 14247, 53342, 315, 52982, 11, 18702, 1778, 46318, 10977, 439, 4149, 91524, 323, 6460, 735, 699, 263, 276, 13, 1102, 574, 2834, 555, 279, 41289, 86805, 38298, 28331, 320, 3791, 37, 8, 304, 2225, 220, 777, 2491, 323,

### Using the Greedy PCO tokenizer to obtain token sets

In [11]:
def get_cpp_counts(domain):
    words = [t for t in open(f'cpp_inputs/words/{domain}.txt','r').read().strip().split(' ')]
    counts = [int(t) for t in open(f'cpp_inputs/counts/{domain}.txt','r').read().strip().split('\n')]
    return {w:c for w,c in zip(words,counts)}

un_counts = get_cpp_counts('un')

In [12]:
# python wrapper for PCO greedy algo
# set token_candidates to look at all possible substrings
# else specify the exact token candidates
token_candidates = set()
greedy_tokenizer = greedy_builder.build_greedy_pco_tokenizer(un_counts, token_candidates)
# initialize graph
greedy_tokenizer.initialize_graph()

Word counts size: 105505
Token set size: 0
Empty token set size selected -> all possible substrings...
Final token set size: 884708
Initial setup phase: 1759 ms


In [13]:
# let's solve for the first 50 steps
tokens, scores = greedy_tokenizer.solve_to_step(50)

Starting main routine...
1. |Ġ [c4 a0 ] | 30035114 | 106 ms | 655 ms | shortlist: 858979
2. |Ġthe [c4 a0 74 68 65 ] | 8286735 | 156 ms | 205 ms | shortlist: 3378
3. |tion [74 69 6f 6e ] | 4043268 | 41 ms | 87 ms | shortlist: 88654
4. |Ġof [c4 a0 6f 66 ] | 3300812 | 45 ms | 47 ms | shortlist: 1045
5. |Ġand [c4 a0 61 6e 64 ] | 3262209 | 36 ms | 37 ms | shortlist: 717
6. |in [69 6e ] | 2782359 | 36 ms | 154 ms | shortlist: 228318
7. |re [72 65 ] | 2384688 | 52 ms | 118 ms | shortlist: 123799
8. |Ġt [c4 a0 74 ] | 2299618 | 50 ms | 64 ms | shortlist: 28821
9. |Ġa [c4 a0 61 ] | 2171690 | 46 ms | 68 ms | shortlist: 47226
10. |er [65 72 ] | 1910147 | 44 ms | 117 ms | shortlist: 149877
11. |en [65 6e ] | 1824071 | 50 ms | 112 ms | shortlist: 121023
12. |Ġco [c4 a0 63 6f ] | 1782132 | 47 ms | 67 ms | shortlist: 37757
13. |it [69 74 ] | 1622191 | 41 ms | 86 ms | shortlist: 79570
14. |Ġw [c4 a0 77 ] | 1404713 | 44 ms | 52 ms | shortlist: 13165
15. |es [65 73 ] | 1365110 | 39 ms | 110 ms | shortlis

In [14]:
print(len(tokens))

50


In [15]:
# add in some manual tokens in between
# useful for warm starts, not that we will have to recalculate the whole cache again after
tokens, scores = greedy_tokenizer.custom_steps(["scar", "edy"])

50. |scar [73 63 61 72 ] | 1602
51. |edy [65 64 79 ] | 5645


In [16]:
print(len(tokens))

52


In [17]:
# continue solving till k=60
tokens, scores = greedy_tokenizer.solve_to_step(60) 

Starting main routine...
53. |Ġe [c4 a0 65 ] | 533333 | 92 ms | 114 ms | shortlist: 26039
54. |il [69 6c ] | 507782 | 42 ms | 64 ms | shortlist: 50519
55. |Ġc [c4 a0 63 ] | 483397 | 43 ms | 57 ms | shortlist: 30385
56. |Ġb [c4 a0 62 ] | 468378 | 40 ms | 51 ms | shortlist: 26087
57. |ly [6c 79 ] | 466512 | 40 ms | 72 ms | shortlist: 85549
58. |th [74 68 ] | 457339 | 48 ms | 60 ms | shortlist: 25726
59. |as [61 73 ] | 457261 | 40 ms | 57 ms | shortlist: 35662
60. |ec [65 63 ] | 453637 | 41 ms | 55 ms | shortlist: 27366
Total time taken: 0 seconds


In [18]:
print(len(tokens))
print(tokens)

60
['Ġ', 'Ġthe', 'tion', 'Ġof', 'Ġand', 'in', 're', 'Ġt', 'Ġa', 'er', 'en', 'Ġco', 'it', 'Ġw', 'es', 'Ġs', 'or', 'at', 'is', 'al', 'Ġp', 'on', 'an', 'Ġin', 'ed', 'Ġto', 'Ġf', 'Ġbe', 'ation', 'ic', 'ou', 'ar', 'ment', 'Ġthat', 'ing', 'Ġdevelop', 'Ġm', 'le', 'Ġh', 'Ġre', 'ĠUnited', 'Ġd', 'Ġcountr', 'st', 'Ġinternational', 'ro', 'ce', 've', 'Ġn', 'Ġwhich', 'scar', 'edy', 'Ġe', 'il', 'Ġc', 'Ġb', 'ly', 'th', 'as', 'ec']
